Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = "Ruchit Patel"
COLLABORATORS = ""

---

# Homework 9: From Expressions to Optimization and Machine Learning

Copyright Luca de Alfaro, 2019-2020. 
License: [CC-BY-NC-ND](https://creativecommons.org/licenses/by-nc-nd/4.0/).


## Submission

[Please submit to this Google Form](https://docs.google.com/forms/d/e/1FAIpQLSdYG6YhWxV5MIgd3yINEOcr2n4wsRv5F1UF2n6PVAIW6J-r9Q/viewform?usp=sf_link).

Deadline: Thursday November 19, 11pm (check on Canvas for updated information).

## Test Format

The test contains 5 questions, for a total of 70 points. 

## ML in a nutshell

Optimization, and machine learning, are intimately connected.  At a very coarse level, ML works as follows. 

First, you come up somehow with a very complicated model $\vec{y} = M(\vec{x}, \vec{\theta})$, which computes an output $\vec{y}$ as a function of an input $\vec{x}$ and of a vector of parameters $\vec{\theta}$.   In general, $\vec{x}$, $\vec{y}$, and $\vec{\theta}$ are vectors, as the model has multiple inputs, multiple outputs, and several parameters.  The model $M$ needs to be complicated, because only complicated models can represent complicated phenomena; for instance, $M$ can be a multi-layer neural net with parameters $\vec{\theta} = [\theta_1, \ldots, \theta_k]$, where $k$ is the number of parameters of the model. 

Second, you come up with a notion of _loss_ $L$, that is, how badly the model is doing.  For instance, if you have a list of inputs $\vec{x}_1, \ldots, \vec{x}_n$, and a set of desired outputs $\vec{y}_1, \ldots, \vec{y}_m$, you can use as loss: 

$$
L(\vec{\theta}) = \sum_{i=1}^n |\!|\vec{y}_i - M(\vec{x}_i, \vec{\theta})|\!| \; .
$$

Here, we wrote $L(\vec{\theta})$ because, once the inputs $\vec{x}_1, \ldots, \vec{x}_n$ and the desired outputs $\vec{y}_1, \ldots, \vec{y}_n$ are chosen, the loss $L$ depends on $\vec{\theta}$. 

Once the loss is chosen, you decrease it, by computing its _gradient_ with respect to $\vec{\theta}$.  Remembering that $\vec{\theta} = [\theta_1, \ldots, \theta_k]$,

$$
\nabla_\vec{\theta} L = \left[ \frac{\partial L}{\partial \theta_1}, \ldots, 
    \frac{\partial L}{\partial \theta_k} \right] \; .
$$

The gradient is a vector that indicates how to tweak $\vec{\theta}$ to decrease the loss.  You then choose a small _step size_ $\delta$, and you update $\vec{\theta}$ via $\vec{\theta} := \vec{\theta} - \delta \nabla_\vec{\theta} L$.  This makes the loss a little bit smaller, and the model a little bit better.  If you repeat this step many times, the model will hopefully get (a good bit) better. 

## Autogradient

The key to _pleasant_ ML is to focus on building the model $M$ in a way that is sufficiently expressive, and on choosing a loss $L$ that is helpful in guiding the optimization.  The computation of the gradient is done automatically for you.  This capability, called _autogradient_, is implemented in ML frameworks such as [Tensorflow](https://www.tensorflow.com), [Keras](https://keras.io), and [PyTorch](https://pytorch.org).  

It is possible to use these advanced ML libraries without ever knowing what is under the hood, and how autogradient works.  Here, we will insted dive in, and implement autogradient.  

Building a model $M$ corresponds to building an expression with inputs $\vec{x}$, $\vec{\theta}$.  We will provide a representaton for expressions that enables both the calculation of the expression value, and the differentiation with respect to any of the inputs.  This will enable us to implement autogradient.  On the basis of this, we will be able to implement a simple ML framework. 

We say we, but we mean you.  _You_ will implement it; we will just provide guidance.

## Expressions with autogradient

Our main task will be to implement a class `Expr` that represents expressions with autogradient.  

### Implementing expressions

We will have `Expr` be the abstract class of a generic expression, and `Plus`, `Multiply`, and so on, be derived classes representing expression with given top-level operators.  The constructor takes the children node.  The code for the constructor, and the code to create addition expressions, is as follows. 



In [ ]:
class Expr(object):

    def __init__(self, *args):
        """Initializes an expression node, with a given list of children
        expressions."""
        self.children = args
        self.value = None # The value of the expression.
        self.values = None # The values of the child expressions.

    def __add__(self, other):
        """Constructs the sum of two expressions."""
        return Plus(self, other)


The code for the `Plus` class, initially, is empty; no `Expr` methods are over-ridden.

In [ ]:
class Plus(Expr):
    """An addition expression."""

    pass


To construct expressions, we need one more thing.  So far, if we write things like `2 + 3`, Python will just consider these as expressions involving numbers, and compute their value.  To write _symbolic_ expressions, we need symbols, or variables.  A variable is a type of expression that just contains a value as child, and that has an `assign` method to assign a value to the variable.  The `assign` method can be used to modify the variable's content (without `assign`, our variables would be constants!). 

In [ ]:
class V(Expr):
    """This class represents a variable.  The derivative rule corresponds
    to d/dx x = 1, but note that it will not be called, since the children
    of a variable are just numbers."""

    def assign(self, v):
        """Assigns a value to the variable."""
        self.children = [v]


This suffices for creating expressions.  Let's create one.

In [ ]:
e = V(3) + 4


## Computing the value of expressions

We now have our first expression.  To compute the expression value, we endow each expression with a method `op`, whose task is to compute the value `self.value` of the expression from the list of values `self.values` of the children.  

Let's implement the `compute` method for an expression.  This method will: 

1. Loop over the children, and computes the list `self.values` of children values as follows: 
    * If the child is an expression (an instance of `Expr`), obtain its value by calling `compute` on it. 
    * If the child is not an instance of `Expr`, then the child must be a number, and we can use its value directly. 
2. Call the method `op` of the expression, to compute `self.value` from `self.values`. 
3. Return `self.value`. 

In [ ]:
class Expr(object):

    def __init__(self, *args):
        """Initializes an expression node, with a given list of children
        expressions."""
        self.children = args
        self.value = None # The value of the expression.
        self.values = None # The values of the child expressions.
        self.gradient = 0 # The value of the gradient.

    def op(self):
        """This operator must be implemented in subclasses; it should
        compute self.value from self.values, thus implementing the
        operator at the expression node."""
        raise NotImplementedError()

    def compute(self):
        """This method computes the value of the expression.
        It first computes the value of the children expressions,
        and then uses self.op to compute the value of the expression."""
        self.values = [e.compute() if isinstance(e, Expr) else e
                        for e in self.children]
        # This computes self.value
        self.op()
        return self.value

    def __repr__(self):
        return ("%s:%r %r (g: %r)" % (
            self.__class__.__name__, self.children, self.value, self.gradient))

    # Expression constructors

    def __add__(self, other):
        return Plus(self, other)

    def __radd__(self, other):
        return Plus(self, other)

    def __sub__(self, other):
        return Minus(self, other)

    def __rsub__(self, other):
        return Minus(other, self)

    def __mul__(self, other):
        return Multiply(self, other)

    def __rmul__(self, other):
        return Multiply(other, self)

    def __truediv__(self, other):
        return Divide(self, other)

    def __rtruediv__(self, other):
        return Divide(other, self)

    def __pow__(self, other):
        return Power(self, other)

    def __rpow__(self, other):
        return Power(other, self)

    def __neg__(self):
        return Negative(self)


Let us give `op` for `Plus`, `Multiply`, and for variables via `V`, so you can see how it works.

In [ ]:
class V(Expr):
    """This class represents a variable."""

    def assign(self, v):
        """Assigns a value to the variable.  Used to fit a model, so we
        can assign the various input values to the variable."""
        self.children = [v]

    def op(self):
        self.value = self.values[0]

    def __repr__(self):
        return "Variable: " + str(self.children[0])


class Plus(Expr):

    def op(self):
        self.value = self.values[0] + self.values[1]

class Multiply(Expr):

    def op(self):
        self.value = self.values[0] * self.values[1]


In [ ]:
#@title Let's define testing helpers.

def check_equal(x, y, msg=None):
    if x != y:
        if msg is None:
            print("Error:")
        else:
            print("Error in", msg, ":")
        print("    Your answer was:", x)
        print("    Correct answer: ", y)
    assert x == y, "%r and %r are different" % (x, y)

def check_almost_equal(x, y, tolerance=0.001, msg=None):
    if abs(x - y) > tolerance:
        if msg is None:
            print("Error:")
        else:
            print("Error in", msg, ":")
        print("    Your answer was:", x)
        print("    Correct answer: ", y)
    assert abs(x - y) < tolerance, "%r and %r are different" % (x, y)

def check_true(x, msg=None):
    if not x:
        if msg is None:
            print("Error: assertion is false")
        else:
            print("Error in", msg, ": false")
    assert x


## Question 1

We will have you implement also multiplication.

In [ ]:
### Exercise: Implement `Multiply`

class Multiply(Expr):
    """A multiplication expression."""

    def op(self):
        # YOUR CODE HERE
        self.value =  self.values[0] * self.values[1]

In [ ]:
# Here you can write additional tests, to help debugging your code.

# YOUR CODE HERE
e = V(0) * 1
print(e.compute())

In [ ]:
### 5 points: Tests for `Multiply`

e = V(2) * 3
check_equal(e.compute(), 6)

e = (V(2) + 3) * V(4)
check_equal(e.compute(), 20)



## Implementing autogradient

The next step consists in implementing autogradient.  Consider an expression $e = E(x_0, \ldots, x_n)$, computed as function of its children expressions $x_0, \ldots, x_n$.  

The goal of the autogradient computation is to accumulate, in each node of the expression, the gradient of the loss with respect to the node's value.  For instance, if the gradient is $2$, we know that if we increase the value of the expression by $\Delta$, then the value of the loss is increased by $2 \Delta$. We accumulate the gradient in the field `self.gradient` of the expression. 

We say _accumulate_ the gradient, because we don't really do:

    self.gradient = ...

Rather, we have a method `e.zero_gradient()` that sets all gradients to 0, and we then _add_ the gradient to this initial value of 0: 

    self.gradient += ... 

We will explain later in detail why we do so; for the moment, just accept it. 

### Computaton of the gradient

In the computation of the autogradient, the expression will receive as input the value $\partial L / \partial e$, where $L$ is the loss, and $e$ the value of the expression.  The quantity $\partial L / \partial e$ is the gradient of the loss with respect to the expression value. 

With this input, the method `compute_gradient` of an Expr $e$ must do the following:

* It must _add_ $\partial L / \partial e$ to the gradient `self.gradient` of the expression. 
* It must call the method `derivate`.  If the expression node has children $c_1, \ldots, c_n$, the method `derivate` must return the list of derivatives of $e$ with respect to its children $x_1, \ldots, x_n$:

  $$
  \left[ \frac{\partial e}{\partial x_1}, \ldots, 
    \frac{\partial e}{\partial x_n} \right]
  $$

  The method `derivate` is implemented not directly in the class `Expr`, but for each specific operator, such as `Plus`, `Multiply`, etc: each operator knows how to compute the derivative of its output value with respect to its children (its arguments).
* It must propagate to each child $x_i$ the gradient 

  $$
  \frac{\partial L}{\partial e} \cdot \frac{\partial e}{\partial x_i} \; , 
  $$

  by calling the method `compute_gradient` of the child with argument $\frac{\partial L}{\partial e} \cdot \frac{\partial e}{\partial x_i}$. 

In [ ]:
def expr_derivate(self): #de/dxi [list of d expression with d children]
    """This method computes the derivative of the operator at the expression
    node.  It needs to be implemented in derived classes, such as Plus,
    Multiply, etc."""
    raise NotImplementedError()

Expr.derivate = expr_derivate

def expr_zero_gradient(self):
    """Sets the gradient to 0, recursively for this expression
    and all its children."""
    self.gradient = 0
    for e in self.children:
        if isinstance(e, Expr):
            e.zero_gradient()

Expr.zero_gradient = expr_zero_gradient

def expr_compute_gradient(self, de_loss_over_de_e=1):
    """Computes the gradient.
    de_loss_over_de_e is the gradient of the output.
    de_loss_over_de_e will be added to the gradient, and then
    we call for each child the method compute_gradient,
    with argument de_loss_over_de_e * d expression / d child.
    The value d expression / d child is computed by self.derivate. """
    pass # We will write this later.

Expr.compute_gradient = expr_compute_gradient


Let us endow our operators `V`, `Plus`, `Multiply` with the `derivate` method, so you can see how it works in practice.

In [ ]:
class V(Expr):
    """This class represents a variable.  The derivative rule corresponds
    to d/dx x = 1, but note that it will not be called, since the children
    of a variable are just numbers."""

    def assign(self, v):
        """Assigns a value to the variable.  Used to fit a model, so we
        can assign the various input values to the variable."""
        self.children = [v]

    def op(self):
        self.value = self.values[0]

    def derivate(self):
        return [1.] # This is not really used.


class Plus(Expr):
    """An addition expression.  The derivative rule corresponds to
    d/dx (x+y) = 1, d/dy (x+y) = 1"""

    def op(self):
        self.value = self.values[0] + self.values[1]

    def derivate(self):
        return [1., 1.]


class Multiply(Expr):
    """A multiplication expression. The derivative rule corresponds to
    d/dx (xy) = y, d/dy(xy) = x"""

    def op(self):
        self.value = self.values[0] * self.values[1]

    def derivate(self):
        return [self.values[1], self.values[0]]


Let us comment on some subtle points, before you get to work at implementing `compute_gradient`.  

**`zero_gradient`:** First, notice how in the implementation of `zero_gradient`, when we loop over the children, we check whether each children is an `Expr` via isinstance(e, Expr).  In general, we have to remember that children can be either `Expr`, or simply numbers, and of course numbers do not have methods such as `zero_gradient` or `compute_gradient` implemented for them. 

**`derivate`:** Second, notice how `derivate` is not implemented in `Expr` directly, but rather, only in the derived classes such as `Plus`.  The derivative of the expression with respect to its arguments depends on which function it is, obviously.  

For `Plus`, we have $e = x_0 + x_1$, and so: 

$$
\frac{\partial e}{\partial x_0} = 1 \qquad \frac{\partial e}{\partial x_1} = 1 \; ,
$$

because $d(x+y)/dx = 1$.  Hence, the `derivate` method of `Plus` returns

$$
\left[ \frac{\partial e}{\partial x_0}, \: \frac{\partial e}{\partial x_1}\right] \; = \; [1, 1] \; .
$$

For `Multiply`, we have $e = x_0 \cdot x_1$, and so:

$$
\frac{\partial e}{\partial x_0} = x_1 \qquad \frac{\partial e}{\partial x_1} = x_0 \; ,
$$

because $d(xy)/dx = y$.  Hence, the `derivate` method of `Plus` returns

$$
\left[ \frac{\partial e}{\partial x_0}, \: \frac{\partial e}{\partial x_1}\right] \; = \; [x_1, x_0] \; .
$$


**Calling `compute` before `compute_gradient`:** Lastly, a very important point: when calling `compute_gradient`, we will assume that `compute` has _already_ been called.  In this way, the value of the expression, and its children, are available for the computation of the gradient.  Note how in `Multiply.derivate` we use these values in order to compute the partial derivatives. 

## Question 2: Implementation of `compute_gradient`

With these clarifications, we ask you to implement the `compute_gradient` method, which again must:

* _add_ $\partial L / \partial e$ to the gradient `self.gradient` of the expression; 
* compute $\frac{\partial e}{\partial x_i}$ for each child $x_i$ by calling the method `derivate` of itself; 
* propagate to each child $x_i$ the gradient $\frac{\partial L}{\partial e} \cdot \frac{\partial e}{\partial x_i}$, by calling the method `compute_gradient` of the child $i$ with argument $\frac{\partial L}{\partial e} \cdot \frac{\partial e}{\partial x_i}$.

Note that you do not need to implement the method `derivate`.  You just need to do the above. 

In [ ]:
### Exercise: Implementation of `compute_gradient`

def expr_compute_gradient(self, de_loss_over_de_e=1):
    """Computes the gradient.
    de_loss_over_de_e is the gradient of the output.
    de_loss_over_de_e will be added to the gradient, and then
    we call for each child the method compute_gradient,
    with argument de_loss_over_de_e * d expression / d child.
    The value d expression / d child is computed by self.derivate. """
    # YOUR CODE HERE
    self.gradient += de_loss_over_de_e
    de_dxi = self.derivate()  
    for i in range(len(de_dxi)):
        if isinstance(self.children[i], Expr):
            self.children[i].compute_gradient(de_loss_over_de_e * de_dxi[i])


Expr.compute_gradient = expr_compute_gradient


In [ ]:
# Here you can write additional tests, to help debugging your code.

# YOUR CODE HERE
vx = V(3)
vz = V(4)
y = vx + vz
y.compute_gradient()

In [ ]:
### 5 Points: Tests for `compute_gradient` for a sum

# First, the gradient of a sum.
vx = V(3)
vz = V(4)
y = vx + vz
check_equal(y.compute(), 7)
y.zero_gradient()
y.compute_gradient()
check_equal(vx.gradient, 1.)


In [ ]:
### 5 Points: Tests for `compute gradient`, with a product. 

vx = V(3)
vz = V(4)
y = vx * vz
check_equal(y.compute(), 12)
y.zero_gradient()
y.compute_gradient()
check_equal(vx.gradient, 4)
check_equal(vz.gradient, 3)


In [ ]:
### 10 points: Finally, the gradient of a function consisting of both sums and products. 

vx = V(1)
vw = V(3)
vz = V(4)
y = (vx + vw) * (vz + 3)
check_equal(y.compute(), 28)
y.zero_gradient()
y.compute_gradient()
check_equal(vx.gradient, 7)
check_equal(vz.gradient, 4)

# And now with a repeated variable. 
vx = V(3)
vw = V(2)
vz = V(5)
y = (vx + vw) * (vz + vx)
check_equal(y.compute(), 40)
y.zero_gradient()
y.compute_gradient()
check_equal(vx.gradient, 13)
check_equal(vz.gradient, 5)


## Why do we accumulate gradients?

We are now in the position of answering the question of why we accumulate gradients.  There are two reasons. 

### Multiple variable occurrence

The most important reason why we need to _add_ to the gradient of each node is that nodes, and in particular, variable nodes, can occur in multiple places in an expression tree.  To compute the total influence of the variable on the expression, we need to _sum_ the influence of each occurrence.  Let's see this with a simple example.  Consider the expression $y = x \cdot x$.  We can code it as follows: 


In [ ]:
vx = V(2.) # Creates a variable vx and initializes it to 2.
y = vx * vx


For $y = x^2$, we have $dy / dx = 2x = 4$, given that $x=2$.  How is this reflected in our code? 

Our code considers separately the left and right occurrences of `vx` in the expression; let us denote them with $vx_l$ and $vx_r$.  The expression can be written as $y = vx_l \cdot vx_r$, and we have that $\partial y / \partial \, vx_l = vx_r = 2$, as $vx_r = 2$.  Similarly, $\partial y / \partial \, vx_r = 2$.  These two gradients are added to `vx.gradient` by the method `compute_gradient`, and we get that the total gradient is 4, as desired.

In [ ]:
y.compute() # We have to call compute() before compute_gradient()
y.zero_gradient()
y.compute_gradient()
print("gradient of vx:", vx.gradient)
check_equal(vx.gradient, 4)


### Multiple data to fit

The other reason why we need to tally up the gradient is that in general, we need to fit a function to more than one data point.  Assume that we are given a set of inputs $x_1, x_2, \ldots, x_n$ and desired outputs $y_1, y_2, \ldots, y_n$. Our goal is to approximate the desired ouputs via an expression 

$$
y = e(x, \theta)
$$

of the input, according to some parameters $\theta$.  Our goal is to choose the parameters $\theta$ to minimize the sum of the square errors for the points: 

$$
L_{tot}(\theta)
= \sum_{i=1}^n L_i(\theta) 
= \sum_{i=1}^n (e(x_i, \theta) - y_i)^2 
\; ,
$$

where $L_i(\theta) = (e(x_i, \theta) - y_i)^2$ is the loss for a single data point.  The gradient $L_{tot}(\theta)$ with respect to $\theta$ can be computed by adding up the gradients for the individual points:

$$
\frac{\partial}{\partial \theta} L_{tot}(\theta) \;=\;
\sum_{i=1}^n \frac{\partial}{\partial \theta} L_i(\theta) \; .
$$

To translate this into code, we will build an expression $e(x, \theta)$ involving the input $x$ and the parameters $\theta$, and an expression 

$$
L = (e(x, \theta) - y)^2
$$

for the loss, involving $x, y$ and $\theta$.  We will then zero all gradients via `zero_gradient`. Once this is done, we compute the loss $L$ for each point $(x_i, y_i)$, and then the gradient $\partial L / \partial \theta$ via a call to `compute_gradient`.  
The gradients for all the points will be added, yielding the gradient for minimizing the total loss over all the points. 

## Question 3: Rounding up the implementation

Now that we have implemented autogradient, as well as the operators `Plus` and `Multiply`, it is time to implement the remaining operators: 

* `Minus`
* `Divide` (no need to worry about division by zero)
* `Power`, representing exponentiation (the `**` operator of Python)
* and the unary minus `Negative`. 

In [ ]:
### Exercise: Implementation of `Minus`, `Divide`, `Power`, and `Negative`

import math

class Minus(Expr):
    """Operator for x - y"""
    #rule (x-y): ret (1, -1)

    def op(self):
        # YOUR CODE HERE
        self.value = self.values[0] - self.values[1]

    def derivate(self):
        # YOUR CODE HERE
        return [1., -1.]

class Divide(Expr):
    """Operator for x / y"""
    # rule dx: 1/y, dy = -x/y^2

    def op(self):
        # YOUR CODE HERE
        self.value = self.values[0]/self.values[1]

    def derivate(self):
        # YOUR CODE HERE
        return [1/self.values[1], (-1 * self.values[0])/(self.values[1] * self.values[1])]

class Power(Expr):
    """Operator for x ** y"""
    #dx: y * x ^ y - 1, dy = x^y * ln(x)

    def op(self):
        # YOUR CODE HERE
        self.value = self.values[0] ** self.values[1]

    def derivate(self):
        # YOUR CODE HERE
        return [self.values[1] * (self.values[0] ** (self.values[1] -1)), (self.values[0] ** self.values[1]) * math.log(self.values[0])]

class Negative(Expr):
    """Operator for -x"""

    def op(self):
        # YOUR CODE HERE
        self.value = -1 * self.values[0]

    def derivate(self):
        # YOUR CODE HERE
        return [-1.]


In [ ]:
# Here you can write additional tests, to help debugging your code.

# YOUR CODE HERE

In [ ]:
### 5 points: Tests for `Minus`

# Minus.
vx = V(3)
vy = V(2)
e = vx - vy
check_equal(e.compute(), 1.)
e.zero_gradient()
e.compute_gradient()
check_equal(vx.gradient, 1)
check_equal(vy.gradient, -1)



In [ ]:
### 10 points: Tests for `Divide`

# Divide.
vx = V(6)
vy = V(2)
e = vx / vy
check_equal(e.compute(), 3.)
e.zero_gradient()
e.compute_gradient()
check_equal(vx.gradient, 0.5)
check_equal(vy.gradient, -1.5)



In [ ]:
### 10 points: Tests for `Power`

# Power.
vx = V(2)
vy = V(3)
e = vx ** vy
check_equal(e.compute(), 8.)
e.zero_gradient()
e.compute_gradient()
check_equal(vx.gradient, 12.)
check_almost_equal(vy.gradient, math.log(2.) * 8.)



In [ ]:
### 5 points: Tests for `Negative`

# Negative
vx = V(6)
e = - vx
check_equal(e.compute(), -6.)
e.zero_gradient()
e.compute_gradient()
check_equal(vx.gradient, -1.)



## Optimization

Let us use our ML framework to fit a parabola to a given set of points.  Here is our set of points:

In [ ]:
points = [
    (-2, 2.7),
    (-1, 3),
    (0, 1.3),
    (1, 2.4),
    (3, 5.5),
    (4, 6.2),
    (5, 9.1),
]


Let us display these points.

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams['figure.figsize'] = (8.0, 3.)
params = {'legend.fontsize': 'large',
          'axes.labelsize': 'large',
          'axes.titlesize':'large',
          'xtick.labelsize':'large',
          'ytick.labelsize':'large'}
matplotlib.rcParams.update(params)

def plot_points(points):
    fig, ax = plt.subplots()
    xs, ys = zip(*points)
    ax.plot(xs, ys, 'r+')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.show()


In [ ]:
plot_points(points)


To fit a parabola to these points, we will build an `Expr` that represents the equation $\hat{y} = ax^2 + bx + c$, where $\hat{y}$ is the value of $y$ predicted by our parabola. 
If $\hat{y}$ is the predicted value, and $y$ is the observed value, to obtain a better prediction of the observations, we minimize the loss $L = (\hat{y} - y)^2$, that is, the square prediction error. 
Written out in detail, our loss is:

$$
    L \;=\; \left( y - \hat{y}\right)^ 2 \;=\; \left( y - (ax^2 + bx + c) \right)^2 \; .
$$

Here, $a, b, c$ are parameters that we need to tune to minimize the loss, and obtain a good fit between the parabola and the points. 
This tuning, or training, is done by repeating the following process many times:

* Zero the gradient
* For each point:
    * Set the values of x, y to the value of the point.
    * Compute the expression giving the loss.
    * Backpropagate.  This computes all gradients with respect to the loss, and in particular, the gradients of the coefficients $a, b, c$. 
* Update the coefficients $a, b, c$ by taking a small step in the direction of the negative gradient (negative, so that the loss decreases).

In [ ]:
### Cell A (see later for why this cell is named)

va = V(0.)
vb = V(0.)
vc = V(0.)
vx = V(0.)
vy = V(0.)

oy = va * vx * vx + vb * vx + vc

loss = (vy - oy) * (vy - oy)


## Question 4: Implementing learning: the `fit` function.

The `fit` function minimizes the loss of fitting a series of $(x, y)$ points with a function.  Precisely, the `fit` function takes as input: 

* An expression `loss`, which gives the loss as a function of the values of two variables, `vx` and `vy`, as well as a list of parameters `params`.  In the above example, `params = [va, vb, vc]`. 
* A list `points` of $(x, y)$-pairs, such as `[(2, 3), (3.2, 4.1)]`. 
* A list `params` of parameters to be optimized.  In the above example, `params = [va, vb, vc]`. 
* A learning-step size `delta`. 
* A number of iterations `num_iterations`.

Below, implement the "for each point" part of the above informal description.  Hint: this takes about 4-5 lines of code.  As a reminder, these are the operations you need to use: 

* For a variable `v`, you can assign it a value `x` by saying `v.assign(x)`.  If you want to assign it a `4`, you do `v.assign(4)`, and so forth (look also at the example at line 13). 
* Forward propagation: for the loss `loss`, you can compute its value via `loss.compute()`. 
* Backward propagation: for the loss `loss`, you can perform gradient backpropagation via `loss.compute_gradient()`. 

In [ ]:
### Exercise: Implementation of `fit`
### Cell B

def fit(loss, points, params, delta=0.0001, num_iterations=4000):
    """
    @param loss: expression giving the loss as a function of variables and parameters.
    @param points: list of (x, y) values to which we have to fit the expression.
    @param params: list of parameters whose value we can tune.
    @param delta: learning step size.
    @param num_iterations: number of learning iterations.
    """

    for iteration_idx in range(num_iterations):
        loss.zero_gradient()
        total_loss = 0.
        for x, y in points:
            # Here, you have to assign values to vx and vy, forward propagate,
            # then backpropagate the gradient.  4 lines are enough; it is not 
            # a trick question.
            # YOUR CODE HERE
            vx.assign(x)
            vy.assign(y)
            loss.compute()
            grad = loss.compute_gradient()
            total_loss += loss.value
        if (iteration_idx + 1) % 100 == 0:
            print("Loss:", total_loss)
        for vv in params:
            vv.assign(vv.value - delta * vv.gradient)
    return total_loss


In [ ]:
# Here you can write additional tests, to help debugging your code.

# YOUR CODE HERE
final_loss = fit(loss, points, [va, vb, vc])

Let's train the coefficients `va`, `vb`, `vc`.  Remember!  Run the cell below after running Cell A and Cell B above.  If you run code below, you need to redefine things by rerunning Cell A and Cell B.  If you find this confusing, reset the kernel and rerun from start to here.  

In [ ]:
### 10 points: if you defined fit correctly, this will converge to less than 2.5 loss. 

final_loss = fit(loss, points, [va, vb, vc])
check_true(final_loss < 2.5)

Let's display the parameter values after the training:

In [ ]:
print("a:", va.value, "b:", vb.value, "c:", vc.value)


Let's display the points, along with the fitted parabola.

In [ ]:
import numpy as np

def plot_points_and_y(points, vx, oy):
    fig, ax = plt.subplots()
    xs, ys = zip(*points)
    ax.plot(xs, ys, 'r+')
    x_min, x_max = np.min(xs), np.max(xs)
    step = (x_max - x_min) / 100
    x_list = list(np.arange(x_min, x_max + step, step))
    y_list = []
    for x in x_list:
        vx.assign(x)
        oy.compute()
        y_list.append(oy.value)
    ax.plot(x_list, y_list)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.show()


In [ ]:
plot_points_and_y(points, vx, oy)


This looks like a good fit!

Let us now show you how to fit a simple linear regression: $y = ax + b$, so $L = (y - (ax + b))^2$. 

In [ ]:
# Parameters
# Sometimes you have to be careful about initial values.
va = V(1.)
vb = V(1.)

# x and y
vx = V(0.)
vy = V(0.)

# Predicted y
oy = va * vx + vb

# Loss
loss = (vy - oy) * (vy - oy)


In [ ]:
fit(loss, points, [va, vb])


In [ ]:
plot_points_and_y(points, vx, oy)


## Question 5: Fitting a 3-rd degree function. 

Using the method illustrated above, fit the following equations to our set of points.  Use `vx`, `xy` for $x$, $y$, and `va`, `vb`, `vc`, `vd` for the parameters.  This is important, or the tests won't pass.

$$
y = a x^3 + b x^2 + c x + d
$$

In [ ]:
### Exercise: fit of y = a^x + bx + c

# Some random initializations.
vx = V(0.)
vy = V(0.)
va = V(-0.1)
vb = V(0.2)
vc = V(-0.3)
vd = V(0.4)
# Define below what is the prediction oy and loss.
# oy = ...
# loss = ...
# YOUR CODE HERE
oy = (va * vx * vx * vx) + (vb *vx *vx) + (vc * vx) +vd
loss = (vy - oy) * (vy - oy)
final_loss = fit(loss, points, [va, vb, vc, vd], delta=2e-5, num_iterations=10000) 

In [ ]:
plot_points_and_y(points, vx, oy)

This is a bit underwhelming: we use a 3rd degree polynomial instead of a 2nd degree one, and yet, the fit is not any better, because the training takes longer and we stopped.  

In [ ]:
### 10 points: convergence, and some sanity checks. 

check_true(final_loss < 3.)
check_almost_equal(va.value, 0, tolerance=0.1)
check_almost_equal(vb.value, 0.43, tolerance=0.1)
check_almost_equal(vc.value, 0.15, tolerance=0.1)
check_almost_equal(vd.value, 1.38, tolerance=0.1)


We see from the small value of `va` that a 3rd degree polynomial, in this case, is not much more useful than a 2nd degree one. 

## Conclusion

Congratulations, you have now implemented the core of [PyTorch](https://pytorch.org)!  Pytorch is build on the ability to write expressions, of which the gradient is computed and automatically propagated; the core constructs, such as `zero_gradient`, `compute_gradient`, and `compute`, are essentially similar.  It is true; Pytorch does add a few things:

* Ability to deal with $n$-dimensional tensors, to represent vectors of inputs and outputs. 
* Optimized backends, using both parallelism and GPUs. 
* Sophisticated mechanisms to deal with large datasets.
* A wide array of operations available on tensors, of which autograd can be computed. 
* Many implemented heuristics to deal with step size.
* Implemented neural layers (sigmoids, ReLU, etc).
* Convolutional nets for images.
* Recurrent nets and LSTMs of streams of data.

And more! But [aside from tensors, backends, GPUs, parallelism, operations, heuristics, neural layers including convolutions and LSTMs, and good output](https://www.youtube.com/watch?v=Y7tvauOJMHo), PyTorch is really nothing more than what you have already implemented.